<a href="https://colab.research.google.com/github/danielAdama/tutorials/blob/main/pytorch_tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. Tensor Basics

In [1]:
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
x = torch.empty(1) # scalar
print(x)
x = torch.empty(3) # vector
print(x)
x = torch.empty(2, 3) # matrix
print(x)
x = torch.empty(2, 2, 3) # tensor, 3 dimensions
print(x)
x = torch.rand(5, 3)
print(x)
x = torch.zeros(5, 3)
print(x)

tensor([9.3030e-29])
tensor([4.3081e-34, 0.0000e+00, 4.3097e-34])
tensor([[4.3160e-34, 0.0000e+00, 4.3156e-34],
        [0.0000e+00, 8.9683e-44, 0.0000e+00]])
tensor([[[4.3123e-34, 0.0000e+00, 4.3124e-34],
         [0.0000e+00, 1.1210e-43, 0.0000e+00]],

        [[1.5695e-43, 0.0000e+00, 4.3123e-34],
         [0.0000e+00, 3.3631e-44, 0.0000e+00]]])
tensor([[0.6371, 0.6541, 0.9364],
        [0.5821, 0.5522, 0.1427],
        [0.9617, 0.3522, 0.3338],
        [0.2813, 0.1654, 0.9484],
        [0.9669, 0.5567, 0.7325]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [4]:
print('size',x.size())
print('shape',x.shape)

size torch.Size([5, 3])
shape torch.Size([5, 3])


In [5]:
x = torch.zeros(5, 3, dtype=torch.float16)
print(x.dtype)

torch.float16


In [6]:
x = torch.rand(5, 3).to(device) # move tensors to GPU device
print(x)

tensor([[0.9918, 0.6505, 0.3650],
        [0.6832, 0.4258, 0.1704],
        [0.8502, 0.1088, 0.6541],
        [0.0905, 0.3669, 0.8387],
        [0.6674, 0.3863, 0.9113]], device='cuda:0')


In [7]:
x = torch.rand(5, 3, device=device) # move tensors to GPU device this method is more efficient

Model Development with Pytorch

In [8]:
import torch.nn as nn

X = torch.tensor([[1], [2], [3], [4], [5], [6], [7], [8]], dtype=torch.float32)
Y = torch.tensor([[1], [2], [3], [4], [5], [6], [7], [8]], dtype=torch.float32)

n_samples, n_features = X.shape
print(n_samples, n_features)


8 1


In [9]:
# Create a test sample
X_test = torch.tensor([[5]], dtype=torch.float32)

In [10]:
# Design the Model to implement the forward pass

# In pytorch, in the __init__ method we define the layers and in the forward method we apply the layers
class LinearRegression(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LinearRegression, self).__init__()
    # Define different layers
    self.lin = nn.Linear(input_dim, output_dim)

  def forward(self, x):
    return self.lin(x)


input_size, output_size = n_features, n_features
model = LinearRegression(input_size, output_size)
print(f"Prediction before training: f({X_test.item()}) = {model(X_test).item():.3f}")

Prediction before training: f(5.0) = -1.881


In [11]:
# Define the loss and optimizer
learning_rate = 0.01
n_epochs = 100
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(n_epochs):
  # predict = forward pass with our model
  y_pred = model(X)

  # loss
  l = loss(Y, y_pred)

  # Calculate the gradient
  l.backward()

  # Update the weights
  optimizer.step()

  # zero the gradients after updating
  optimizer.zero_grad()

  if (epoch+1) % 10 == 0:
    w, b = model.parameters() # unpack parameters
    print('epoch ', epoch+1, ': w = ', w[0][0].item(), ' loss = ', l.item())

print(f"Prediction after training: f({X_test.item()}) = {model(X_test).item():.3f}")

epoch  10 : w =  0.8462545871734619  loss =  0.15362590551376343
epoch  20 : w =  0.8530244827270508  loss =  0.14174726605415344
epoch  30 : w =  0.8587883114814758  loss =  0.13084836304187775
epoch  40 : w =  0.864325761795044  loss =  0.12078744173049927
epoch  50 : w =  0.8696460723876953  loss =  0.11150017380714417
epoch  60 : w =  0.874757707118988  loss =  0.10292690992355347
epoch  70 : w =  0.8796689510345459  loss =  0.09501292556524277
epoch  80 : w =  0.8843875527381897  loss =  0.08770741522312164
epoch  90 : w =  0.8889211416244507  loss =  0.0809636265039444
epoch  100 : w =  0.8932769298553467  loss =  0.07473837584257126
Prediction after training: f(5.0) = 5.066


Neural Network

In [19]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
input_size = 784 # 28 x 28
hidden_size = 500
num_classes = 10
num_epochs = 20
batch_size = 100
learning_rate = 0.001


# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=False, 
                                           transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)

# examples = iter(test_loader)
# example_data, example_targets = examples.next()

# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.imshow(example_data[i][0], cmap='gray')
# plt.show()

In [20]:
from torch.utils.data import dataloader
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass and loss calculation
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/20], Step [100/600], Loss: 0.3745
Epoch [1/20], Step [200/600], Loss: 0.1726
Epoch [1/20], Step [300/600], Loss: 0.2418
Epoch [1/20], Step [400/600], Loss: 0.2344
Epoch [1/20], Step [500/600], Loss: 0.1894
Epoch [1/20], Step [600/600], Loss: 0.1803
Epoch [2/20], Step [100/600], Loss: 0.1054
Epoch [2/20], Step [200/600], Loss: 0.0834
Epoch [2/20], Step [300/600], Loss: 0.0833
Epoch [2/20], Step [400/600], Loss: 0.1536
Epoch [2/20], Step [500/600], Loss: 0.0989
Epoch [2/20], Step [600/600], Loss: 0.0987
Epoch [3/20], Step [100/600], Loss: 0.0534
Epoch [3/20], Step [200/600], Loss: 0.0543
Epoch [3/20], Step [300/600], Loss: 0.0455
Epoch [3/20], Step [400/600], Loss: 0.0807
Epoch [3/20], Step [500/600], Loss: 0.0415
Epoch [3/20], Step [600/600], Loss: 0.1135
Epoch [4/20], Step [100/600], Loss: 0.0674
Epoch [4/20], Step [200/600], Loss: 0.0370
Epoch [4/20], Step [300/600], Loss: 0.1066
Epoch [4/20], Step [400/600], Loss: 0.0277
Epoch [4/20], Step [500/600], Loss: 0.0237
Epoch [4/20

In [21]:
# Test the model: we don't need to compute gradients
with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)

    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)

        # max returns (output_value ,index)
        _, predicted = torch.max(outputs, 1)
        n_correct += (predicted == labels).sum().item()

    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')

Accuracy of the network on the 10000 test images: 98.09 %


## 5. CNN

This section covers:

- Convolutional Layers
- MaxPooling
- Save/Load model

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
num_epochs = 20
batch_size = 32
learning_rate = 0.001

# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(imgs):
    imgs = imgs / 2 + 0.5   # unnormalize
    npimgs = imgs.numpy()
    plt.imshow(np.transpose(npimgs, (1, 2, 0)))
    plt.show()

# one batch of random training images
# dataiter = iter(train_loader)
# images, labels = dataiter.next()
# img_grid = torchvision.utils.make_grid(images[0:25], nrow=5)
# imshow(img_grid)

Files already downloaded and verified
Files already downloaded and verified


In [24]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64*4*4, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # N, 3, 32, 32
        x = F.relu(self.conv1(x))   # -> N, 32, 30, 30
        x = self.pool(x)            # -> N, 32, 15, 15
        x = F.relu(self.conv2(x))   # -> N, 64, 13, 13
        x = self.pool(x)            # -> N, 64, 6, 6
        x = F.relu(self.conv3(x))   # -> N, 64, 4, 4
        x = torch.flatten(x, 1)     # -> N, 1024
        x = F.relu(self.fc1(x))     # -> N, 64
        x = self.fc2(x)             # -> N, 10
        return x


model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):

    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

    print(f'[{epoch + 1}] loss: {running_loss / n_total_steps:.3f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

[1] loss: 1.469
[2] loss: 1.091
[3] loss: 0.928
[4] loss: 0.821
[5] loss: 0.747
[6] loss: 0.689
[7] loss: 0.646
[8] loss: 0.605
[9] loss: 0.564
[10] loss: 0.530
[11] loss: 0.497
[12] loss: 0.469
[13] loss: 0.439
[14] loss: 0.411
[15] loss: 0.384
[16] loss: 0.367
[17] loss: 0.341
[18] loss: 0.322
[19] loss: 0.304
[20] loss: 0.281
Finished Training


In [25]:
loaded_model = ConvNet()
loaded_model.load_state_dict(torch.load(PATH)) # it takes the loaded dictionary, not the path file itself
loaded_model.to(device)
loaded_model.eval()

with torch.no_grad():
    n_correct = 0
    n_correct2 = 0
    n_samples = len(test_loader.dataset)

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_correct += (predicted == labels).sum().item()

        outputs2 = loaded_model(images)
        _, predicted2 = torch.max(outputs2, 1)
        n_correct2 += (predicted2 == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the model: {acc} %')

    acc = 100.0 * n_correct2 / n_samples
    print(f'Accuracy of the loaded model: {acc} %')

Accuracy of the model: 71.39 %
Accuracy of the loaded model: 71.39 %
